In [ ]:
#import the depedencies

import torch
import torch.nn as nn
from torch import optim, as_tensor
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
from torchvision import transforms, datasets,models


import cv2 as cv
import numpy as np
from PIL import Image
from pdb import set_trace
import time
import copy

from pathlib import Path
import os
import sys
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage import io,transform
from tqdm import trange,tqdm
import csv
import glob
import dlib


In [ ]:
#transformation on the imags

data_transform ={
    'train':transforms.Compose([
                                transforms.RandomHorizontalFlip(),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485,0.456,0.406],
                                                     [0.229,0.224,0.225])
    ]),
    'validation':transforms.Compose([
                                     transforms.ToTensor(),
                                     transforms.Normalize([0.485,0.456,0.406],
                                                          [0.229,0.224,0.225])
    ]),
}

In [ ]:
root = '/content/drive/MyDrive/face_recognition_dataset/'
data = datasets.ImageFolder(root=root+'train',transform=data_transform['train'])
dataloader = torch.utils.data.DataLoader(data,batch_size=20,shuffle=True)

valid_data = datasets.ImageFolder(root=root+'test',transform=data_transform['validation'])
valid_loader = torch.utils.data.DataLoader(valid_data,batch_size=20,shuffle=True)

images, labels = next(iter(dataloader))
print(images[1].shape)
print(labels[1])



In [ ]:
## To show a Tensor Image
import torchvision
def imshow(img,title=None):
  img = img.numpy().transpose((1,2,0))
  mean = np.array([0.485,0.456,0.406])
  std = np.array([0.229,0.224,0.225])
  img = std*img+mean
  img = np.clip(img,0,1)
  plt.imshow(img)
  if title is not None:
    plt.title(title)
  plt.pause(0.001)

inputs,classes = next(iter(dataloader))
out = torchvision.utils.make_grid(inputs)

imshow(out)

In [ ]:
!pip3 install facenet_pytorch
from facenet_pytorch import InceptionResnetV1

model = InceptionResnetV1(pretrained='vggface2',num_classes=4)

In [ ]:
list(model.children())[-6:]

In [ ]:
layer_list = list(model.children())[-5:]
layer_list

In [ ]:
model = nn.Sequential(*list(model.children())[:-5])

In [ ]:
for param in model.parameters():
  param.requires_grad = False

In [ ]:
class Flatten(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self,x):
    x = x.view(x.size(0),-1)
    return x

class normalize(nn.Module):
  def __init__(self):
    super(normalize,self).__init__()

  def forward(self,x):
    x = F.normalize(x,p=2,dim=1)
    return x

In [ ]:
model.avgpool_1a = nn.AdaptiveAvgPool2d(output_size=1)
model.last_linear = nn.Sequential(Flatten(),
                                  nn.Linear(1792,512,bias=False),
                                  normalize()
                                  )

model.logits = nn.Linear(layer_list[4].in_features,4)
model.softmax = nn.Softmax(dim=1)

In [ ]:
print(model)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=1e-2)

In [ ]:
##Training the model

epochs = 40
training_loss = []
validation_loss = []
model.to(device)
min_val_loss = np.Inf



for e in range(epochs):
  train_loss=0
  valid_loss = 0
  accuracy = []
  model.train()
  for images, labels in dataloader:
    images,labels = images.to(device), labels.to(device)
    optimizer.zero_grad()
    output = model(images)
    loss = criterion(output,labels)
    loss.backward()
    optimizer.step()
    train_loss += loss.item()
  
  model.eval()
  for images, labels in valid_loader:
    images,labels = images.to(device),labels.to(device)
    output = model(images)
    _,preds = torch.max(output,1)
    equals = torch.sum(preds==labels.data)
    accuracy.append(equals)
    loss = criterion(output,labels)
    valid_loss += loss.item()

  print(f'{e+1}/{epochs}... Training Loss: {train_loss/len(dataloader)}... Validation_Loss: {valid_loss/len(valid_loader)}... Accuracy: {sum(accuracy)/len(accuracy)}')
  if min_val_loss >= valid_loss:
    min_val_loss = valid_loss
    torch.save(model.state_dict(),'model_parameters.pth')
  
  training_loss.append(train_loss)
  validation_loss.append(valid_loss)


  

1/40... Training Loss: 0.8857806568795984... Validation_Loss: 1.0343634172489768... Accuracy: 13.263157844543457
2/40... Training Loss: 0.8829647871580991... Validation_Loss: 1.030483205067484... Accuracy: 13.263157844543457
3/40... Training Loss: 0.8933218893679705... Validation_Loss: 1.049581163807919... Accuracy: 13.210526466369629
4/40... Training Loss: 0.8908774744380604... Validation_Loss: 1.0153805237067373... Accuracy: 13.263157844543457
5/40... Training Loss: 0.8793164112351157... Validation_Loss: 1.0121090443510758... Accuracy: 13.315789222717285
6/40... Training Loss: 0.8821388483047485... Validation_Loss: 1.0321674566519887... Accuracy: 13.263157844543457
7/40... Training Loss: 0.8827980959957297... Validation_Loss: 1.0182586626002663... Accuracy: 13.315789222717285
8/40... Training Loss: 0.8743568428538062... Validation_Loss: 1.0237299956773456... Accuracy: 13.263157844543457
9/40... Training Loss: 0.8714182119477879... Validation_Loss: 1.0162356746824164... Accuracy: 13.3

In [ ]:
print(valid_loader.__len__())

19


In [ ]:
images, labels = next(iter(valid_loader))
images,labels = images.to(device), labels.to(device)
output = model(images)
_,preds = torch.max(output,1)
equals = torch.sum(preds==labels.data)
val = equals.item()/len(labels)
print(val)
print(labels)

0.5
tensor([3, 1, 0, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 0, 0, 1, 3, 2, 3, 1],
       device='cuda:0')


In [ ]:
accuracy = []
for images, labels in valid_loader:
  images, labels = images.to(device), labels.to(device)
  output = model(images)
  _, preds = torch.max(output,1)
  equals = torch.sum(preds==labels.data)
  val = equals.item()/len(labels)
  print(val)
  accuracy.append(val)

print(sum(accuracy)/len(accuracy))
  

0.7
0.75
0.75
0.65
0.8
0.75
0.9
0.85
0.6
0.75
0.65
0.7
0.75
0.6
0.9
0.55
0.7
0.6
0.6666666666666666
0.7166666666666666


In [ ]:

output = model(images[1].unsqueeze(dim=0))
print(torch.max(output,1))

torch.return_types.max(
values=tensor([0.7867], device='cuda:0', grad_fn=<MaxBackward0>),
indices=tensor([3], device='cuda:0'))


In [ ]:
PATH = "/content/drive/MyDrive/entire_model2.pt"

# Save
torch.save(model.state_dict(), PATH)

# Load
model = torch.load(PATH)


AttributeError: ignored

In [ ]:
import pickle

filename = 'model_1.0.pth'

pickle.dump(model, open(filename, 'wb'))

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
print(loaded_model.forward(images[1].unsqueeze(1)))

RuntimeError: ignored

In [ ]:
model.to(device)

AttributeError: ignored

In [ ]:
device='cpu'

loaded_model.to(device)

Sequential(
  (0): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (1): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (2): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (5): BasicConv2d(
    (conv): Conv2d(80, 192, kernel_size=(3,

In [ ]:
pickle.dump(loaded_model, open('model_2.0.pth', 'wb'))